# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [82]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-ec24a79924-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Disney explorative search") 

Consider the following exploratory information need:

> investigate films produced by Disney and compare workers among films.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Walt Disney Animation Studios

3. Identify the BGP for Walt Disney (person)

4. List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

5. Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

6. Identify the BGP for the film "The Little Mermaid"

7. Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label). 

8. Identify the BGP for Academy Awards

9. For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [85]:
queryString = """
SELECT DISTINCT ?instance ?name ?subclass ?subclassname ?subclass1 ?subclassname1
WHERE {
    wd:Q36479 wdt:P31 ?instance.
    ?instance sc:name ?name.
    ?instance wdt:P279 ?subclass.
    ?subclass sc:name ?subclassname .
    ?subclass wdt:P279 ?subclass1 .
    ?subclass1 sc:name ?subclassname1 .

}

"""

print("Results")

x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q29168811'), ('name', 'animated feature film'), ('subclass', 'http://www.wikidata.org/entity/Q202866'), ('subclassname', 'animated film'), ('subclass1', 'http://www.wikidata.org/entity/Q11424'), ('subclassname1', 'film')]
[('instance', 'http://www.wikidata.org/entity/Q29168811'), ('name', 'animated feature film'), ('subclass', 'http://www.wikidata.org/entity/Q24869'), ('subclassname', 'feature film'), ('subclass1', 'http://www.wikidata.org/entity/Q11424'), ('subclassname1', 'film')]
2


Final query for this task

In [88]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q11424 ?p ?obj .
   ?p sc:name ?name.
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p',

## Task 2
Identify the BGP for Walt Disney Animation Studios

In [87]:
queryString = """
SELECT DISTINCT ?n1 ?name
WHERE {
    wd:Q36479 wdt:P272 ?n1.
    ?n1 sc:name ?name

}

"""

print("Results")
x=run_query(queryString)

Results
[('n1', 'http://www.wikidata.org/entity/Q191224'), ('name', 'Walt Disney Pictures')]
[('n1', 'http://www.wikidata.org/entity/Q1047410'), ('name', 'Walt Disney Animation Studios')]
2


Final query for this task

In [89]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q1047410 ?p ?obj.
    ?p sc:name ?name

} LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('name', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/

## Task 3
Identify the BGP for Walt Disney (person)

In [90]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
    ?person wdt:P31 wd:Q5 .
    ?person sc:name ?name.
    FILTER (str(?name) = 'Walt Disney')
}

LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q8704'), ('name', 'Walt Disney')]
1


Final query for this task

In [91]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q8704 ?p ?obj .
   ?p sc:name ?name.
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5622'), ('name', 'National Cartoonists Society member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata

## Task 4
List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

Explanation - By using the Q-value for 'Walt Disney' in Task 3, In query1 finding the date of birth and date of death of Walt Disney. In query2 by the production company property found in Task2, finding the films created by Walt Disney Animation Studios.By using these data in final query we find the films created while Wlat Disney was alive.

In [94]:
queryString = """
SELECT DISTINCT ?dob ?dod ?year ?year1
WHERE {
    wd:Q8704 wdt:P569 ?dob .
    wd:Q8704 wdt:P570 ?dod.
    BIND(STRBEFORE( STR(?dob), "-" )  AS ?year)
    BIND(STRBEFORE( STR(?dod), "-" )  AS ?year1)
}

LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('dob', '1901-12-05T00:00:00Z'), ('dod', '1966-12-15T00:00:00Z'), ('year', '1901'), ('year1', '1966')]
1


Final query for this task

In [96]:
queryString = """
SELECT DISTINCT ?movies ?name ?date
WHERE {
    ?movies wdt:P272 wd:Q1047410.
    ?movies sc:name ?name .
    ?movies wdt:P577 ?d .
    BIND(STRBEFORE( STR(?d), "-" )  AS ?date)
    FILTER(str(?date) > '1901' && str(?date) < '1966' )

} LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('movies', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty'), ('date', '1959')]
[('movies', 'http://www.wikidata.org/entity/Q3236849'), ('name', "Let's Stick Together"), ('date', '1952')]
[('movies', 'http://www.wikidata.org/entity/Q842306'), ('name', 'Saludos Amigos'), ('date', '1942')]
[('movies', 'http://www.wikidata.org/entity/Q2090328'), ('name', 'Trick or Treat'), ('date', '1952')]
[('movies', 'http://www.wikidata.org/entity/Q640679'), ('name', 'Rugged Bear'), ('date', '1953')]
[('movies', 'http://www.wikidata.org/entity/Q2399982'), ('name', 'Tea for Two Hundred'), ('date', '1948')]
[('movies', 'http://www.wikidata.org/entity/Q3028329'), ('name', 'Two Weeks Vacation'), ('date', '1952')]
[('movies', 'http://www.wikidata.org/entity/Q3520106'), ('name', 'The Brave Engineer'), ('date', '1950')]
[('movies', 'http://www.wikidata.org/entity/Q2700743'), ('name', 'Clock Cleaners'), ('date', '1937')]
[('movies', 'http://www.wikidata.org/entity/Q3521221'), ('na

## Task 5
Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

In [99]:
queryString = """
SELECT DISTINCT ?vactor ?actorname 
WHERE {
   ?movies wdt:P272 wd:Q1047410.
    ?movies sc:name ?name .
    ?movies wdt:P725 ?vactor .
    ?vactor sc:name ?actorname .
} GROUP BY ?actorname ?vactor


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('vactor', 'http://www.wikidata.org/entity/Q430625'), ('actorname', 'Taylor Holmes')]
[('vactor', 'http://www.wikidata.org/entity/Q3376737'), ('actorname', 'Peter Mansbridge')]
[('vactor', 'http://www.wikidata.org/entity/Q984077'), ('actorname', 'Nick Swardson')]
[('vactor', 'http://www.wikidata.org/entity/Q3181202'), ('actorname', 'Lauren MacMullan')]
[('vactor', 'http://www.wikidata.org/entity/Q229728'), ('actorname', 'Sarah Chalke')]
[('vactor', 'http://www.wikidata.org/entity/Q328456'), ('actorname', 'Jennifer Darling')]
[('vactor', 'http://www.wikidata.org/entity/Q740812'), ('actorname', 'Adam Ryen')]
[('vactor', 'http://www.wikidata.org/entity/Q229230'), ('actorname', 'Evan Rachel Wood')]
[('vactor', 'http://www.wikidata.org/entity/Q707734'), ('actorname', 'Bernard Fox')]
[('vactor', 'http://www.wikidata.org/entity/Q16296'), ('actorname', 'Patrick Stewart')]
[('vactor', 'http://www.wikidata.org/entity/Q178166'), ('actorname', 'Vin Diesel')]
[('vactor', 'http://www.wikida

Final query for this task

In [ ]:
queryString = """
SELECT DISTINCT ?vactor ?actorname (COUNT(?movies) AS ?moviescount)
WHERE {
   ?movies wdt:P272 wd:Q1047410.
    ?movies sc:name ?name .
    ?movies wdt:P725 ?vactor .
    ?vactor sc:name ?actorname .
} GROUP BY ?actorname ?vactor
HAVING (count(?actorname) > 5)


LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('vactor', 'http://www.wikidata.org/entity/Q349350'), ('actorname', 'Alan Tudyk'), ('moviescount', '7')]
[('vactor', 'http://www.wikidata.org/entity/Q310292'), ('actorname', 'Jim Cummings'), ('moviescount', '10')]
[('vactor', 'http://www.wikidata.org/entity/Q296577'), ('actorname', 'Frank Welker'), ('moviescount', '7')]
[('vactor', 'http://www.wikidata.org/entity/Q178882'), ('actorname', 'Kristen Bell'), ('moviescount', '6')]
[('vactor', 'http://www.wikidata.org/entity/Q264418'), ('actorname', 'Russi Taylor'), ('moviescount', '6')]
5


## Task 6
Identify the BGP for the film "The Little Mermaid"

In [102]:
queryString = """
SELECT DISTINCT ?movies ?name
WHERE {
    ?movies wdt:P31 wd:Q29168811 .
    ?movies sc:name ?name.
    FILTER (str(?name) = 'The Little Mermaid')
}

LIMIT 50
"""


print("Results")
x=run_query(queryString)

Results
[('movies', 'http://www.wikidata.org/entity/Q190135'), ('name', 'The Little Mermaid')]
1


Final query for this task

In [103]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q190135 ?p ?obj .
   ?p sc:name ?name.
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), (

## Task 7
Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label).

In [106]:
queryString = """
SELECT DISTINCT ?obj ?tlkcrew ?object ?tlmcrew
WHERE {
   wd:Q36479 wdt:P3092 ?obj .
   ?obj sc:name ?tlkcrew.
   wd:Q190135 wdt:P3092 ?object .
   ?object sc:name ?tlmcrew .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16204825'), ('tlkcrew', 'Barry Kooser'), ('object', 'http://www.wikidata.org/entity/Q28803503'), ('tlmcrew', 'Robert Stanton')]
[('obj', 'http://www.wikidata.org/entity/Q19822599'), ('tlkcrew', 'Robert Walker'), ('object', 'http://www.wikidata.org/entity/Q28803503'), ('tlmcrew', 'Robert Stanton')]
[('obj', 'http://www.wikidata.org/entity/Q28803503'), ('tlkcrew', 'Robert Stanton'), ('object', 'http://www.wikidata.org/entity/Q28803503'), ('tlmcrew', 'Robert Stanton')]
[('obj', 'http://www.wikidata.org/entity/Q28813470'), ('tlkcrew', 'Vera Pacheco'), ('object', 'http://www.wikidata.org/entity/Q28803503'), ('tlmcrew', 'Robert Stanton')]
[('obj', 'http://www.wikidata.org/entity/Q28855009'), ('tlkcrew', 'Ric Sluiter'), ('object', 'http://www.wikidata.org/entity/Q28803503'), ('tlmcrew', 'Robert Stanton')]
[('obj', 'http://www.wikidata.org/entity/Q28859357'), ('tlkcrew', 'Jeff Dickson'), ('object', 'http://www.wikidata.org/entity/Q28803503'), (

Final query for this task

In [107]:
queryString = """
SELECT DISTINCT ?obj ?tlkcrew
WHERE {
   wd:Q36479 wdt:P3092 ?obj .
   ?obj sc:name ?tlkcrew.
   wd:Q190135 wdt:P3092 ?object .
   ?object sc:name ?tlmcrew .
   FILTER(?tlkcrew = ?tlmcrew)
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28803503'), ('tlkcrew', 'Robert Stanton')]
[('obj', 'http://www.wikidata.org/entity/Q28813470'), ('tlkcrew', 'Vera Pacheco')]
[('obj', 'http://www.wikidata.org/entity/Q28802729'), ('tlkcrew', 'Philip Phillipson')]
[('obj', 'http://www.wikidata.org/entity/Q28812897'), ('tlkcrew', 'Greg Drolette')]
[('obj', 'http://www.wikidata.org/entity/Q3242412'), ('tlkcrew', 'Lisa Keene')]
[('obj', 'http://www.wikidata.org/entity/Q28812894'), ('tlkcrew', 'Doug Ball')]
[('obj', 'http://www.wikidata.org/entity/Q7376523'), ('tlkcrew', 'Rubén Procopio')]
[('obj', 'http://www.wikidata.org/entity/Q28870524'), ('tlkcrew', 'Bill Berg')]
[('obj', 'http://www.wikidata.org/entity/Q28812892'), ('tlkcrew', 'Kathy Altieri')]
[('obj', 'http://www.wikidata.org/entity/Q27567832'), ('tlkcrew', 'Brian Clift')]
[('obj', 'http://www.wikidata.org/entity/Q28812742'), ('tlkcrew', 'Lorenzo E. Martinez')]
[('obj', 'http://www.wikidata.org/entity/Q5218823'), ('tlkcrew', 'Daniel

## Task 8
Identify the BGP for Academy Awards

In [108]:
queryString = """
SELECT DISTINCT  ?instance ?name1
WHERE {
   wd:Q8704 wdt:P166 ?obj .
   ?obj sc:name ?name.
   FILTER REGEX(?name, "Academy Award") 
   ?obj wdt:P31 ?instance .
   ?instance sc:name ?name1 . 
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q96474688'), ('name1', 'award for best film by genre')]
[('instance', 'http://www.wikidata.org/entity/Q96474697'), ('name1', 'award for best short film by genre')]
[('instance', 'http://www.wikidata.org/entity/Q19020'), ('name1', 'Academy Awards')]
[('instance', 'http://www.wikidata.org/entity/Q96474696'), ('name1', 'award for best short film')]
4


Final query for this task

In [109]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q19020 ?p ?obj.
    ?p sc:name ?name

} LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1407'), ('name', 'MusicBrainz series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003

## Task 9
For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

Explanation - Query1 ?movies get the IRI of all the films created by the Walt Disney Animation Studios, ?award get the IRI or all the nominations to the films.In Query2 by using Query1 data filtering the awards using the REGEX to find the nomination for 'Academy Award' and counting the number of nomination.Final query gives the films name and the number of nomination

In [114]:
queryString = """
SELECT DISTINCT ?movies ?name ?award ?awardname
WHERE {
   ?movies wdt:P272 wd:Q1047410.
    ?movies sc:name ?name .
    ?movies wdt:P1411 ?award .
    ?award sc:name ?awardname .
    FILTER REGEX(?awardname, "Academy Award") 
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movies', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('award', 'http://www.wikidata.org/entity/Q106800'), ('awardname', 'Academy Award for Best Animated Feature')]
[('movies', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast'), ('award', 'http://www.wikidata.org/entity/Q488651'), ('awardname', 'Academy Award for Best Original Score')]
[('movies', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast'), ('award', 'http://www.wikidata.org/entity/Q102427'), ('awardname', 'Academy Award for Best Picture')]
[('movies', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast'), ('award', 'http://www.wikidata.org/entity/Q830079'), ('awardname', 'Academy Award for Best Sound')]
[('movies', 'http://www.wikidata.org/entity/Q379873'), ('name', 'Who Framed Roger Rabbit'), ('award', 'http://www.wikidata.org/entity/Q393686'), ('awardname', 'Academy Award for Best Visual Effects')]
[('movies', 'http://www.wi

Final query for this task

In [116]:
queryString = """
SELECT DISTINCT ?movies ?name (COUNT(?name) AS ?mcount)
WHERE {
   ?movies wdt:P272 wd:Q1047410.
    ?movies sc:name ?name .
    ?movies wdt:P1411 ?award .
    ?award sc:name ?awardname .
    FILTER REGEX(?awardname, "Academy Award") 
} ORDER BY ?name

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movies', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast'), ('mcount', '3')]
[('movies', 'http://www.wikidata.org/entity/Q13091172'), ('name', 'Big Hero 6'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q212792'), ('name', 'Bolt'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q16857388'), ('name', 'Feast'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q246283'), ('name', 'Frozen'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q12124484'), ('name', 'Get a Horse!'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q3259450'), ('name', 'Lorenzo'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q2252695'), ('name', 'Paperman'), ('mcount', '1')]
[('movies', 'http://www.wikidata.org/entity/Q640679'), ('name', 'Rugged Bear'), ('mcount', '1')]
[('movies', 'http://www.wikidata.o